## TOC:
* [Parameters](#parameters)
* [Elliptic Curve Mathematics](#ec-math)
    * [Modular Inverse](#modular-inverse)
    * [Double](#double)
    * [Add](#add)
    * [Multiply](#multiply)
        * [Double-and-add algorithm](#double-add)
* [ECDSA](#ecdsa)
    * [Key Generation](#key-gen)
    * [Sign](#sign)
    * [Verify](#verity)


In [271]:
from IPython.display import clear_output
from time import sleep, time
from hashlib import sha256
from secrets import randbelow

***
## Parameters<a class="anchor" id="parameters"></a>

The following variables are the ones used in secp256k1, the elliptic curve used in Bitcoin.</br></br>
Elliptic curve fuction:   $y^2 = x^3 + ax + b$

In [272]:
a = 0
b = 7

# prime field
p = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1

# order
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

G = [# genarator point
  55066263022277343669578718895168534326250603453777594175500187360389116729240,
  32670510020758816978083085130507043184471273380659243275938904335757337482424
]

***
## Elliptic Curve Mathematics<a class="anchor" id="ec-math"></a>


### Modular Inverse<a class="anchor" id="modular-inverse"></a>

Extended Euclidean algorithm

In [273]:
def inverse(a, m = p):
    m_orig = m
    if a == 0:
        raise ZeroDivisionError(f"Variable a={a}; should be bigger than 0.")
    a = a % m
    prevy, y = 0, 1
    while a > 1:
        q = m // a
        y, prevy = prevy - q * y, y
        a, m = m % a, a
    return y % m_orig

print(inverse(13, 47))
print(pow(13, -1, 47)) # pyhton3.8+ and above 

29
29


In [274]:
def animation(x, y, m):

    for i in range(x, x*y+1):
        print("Finite Fields: Multiplication")
        pos = i % m

        print(" "+" "*pos+str(pos))
        print("0"+" "*pos+"\u21E9"+" "*(m-pos-1)+str(m))
        print("|"+"\u2022"*m+"|")


        print("\n{} * {} = {}".format(x, y,str(pos)if i ==x*y else ""))
        sleep(.05)
        clear_output(wait=True)
    result = pos
    sleep(1.2)
    for i in range(result + 1 ,(result-1)*pow(x, -1, m)+2):
        print("Finite Fields: Multiplication (Find Inverse)")
        pos = i % m
        times = i // m

        print(" "+" "*pos+str(pos))
        print("0"+" "*pos+"\u21E9"+" "*(m-pos-1)+str(m))
        print("|"+"\u2022"*m+"|")


        print("\n{} * {} = {}".format(x, y, result))
        print("\n{} * {} = {}".format(result, times, i%m))
        sleep(.05)
        clear_output(wait=True)


#animation(8, 13, 47)

### Double<a class="anchor" id="double"></a>

Used in case the two points are the same. We calculate the tangent to the point and find out where else does intersect the curve.

$slope = (3x₁² + a) / 2y₁$</br>
$x = slope² - 2x₁$</br>
$y = slope * (x₁ - x) - y₁$

In [275]:
def double(point):
    slope = ((3 * point[0] ** 2 + a) * inverse((2 * point[1]), p)) % p
    x = (slope ** 2 - (2 * point[0])) % p
    y = (slope * (point[0] - x) - point[1]) % p
    return (x, y)

result = double(G)
print(f"double(G): {{\n\tx: {result[0]},\n\ty: {result[1]}\n}}")

double(G): {
	x: 89565891926547004231252920425935692360644145829622209833684329913297188986597,
	y: 12158399299693830322967808612713398636155367887041628176798871954788371653930
}


### Add<a class="anchor" id="add"></a>

Used when the points are different.

$slope = (y₁ - y₂) / (x₁ - x₂)$</br>
$x = slope² - x₁ - x₂$</br>
$y = slope * (x₁ - x) - y₁$

In [276]:
def add(point1, point2):
  if point1 == point2:
    return double(point1)
  slope = ((point1[1] - point2[1]) * inverse(point1[0] - point2[0], p)) % p
  x = (slope ** 2 - point1[0] - point2[0]) % p
  y = ((slope * (point1[0] - x)) - point1[1]) % p
  return (x, y)

T = add(G, G)
print(f"T: {{\n\tx: {T[0]},\n\ty: {T[1]}\n}}")
T2 = add(T, G)
print(f"T2: {{\n\tx: {T2[0]},\n\ty: {T2[1]}\n}}")

T: {
	x: 89565891926547004231252920425935692360644145829622209833684329913297188986597,
	y: 12158399299693830322967808612713398636155367887041628176798871954788371653930
}
T2: {
	x: 112711660439710606056748659173929673102114977341539408544630613555209775888121,
	y: 25583027980570883691656905877401976406448868254816295069919888960541586679410
}


### Multiply<a class="anchor" id="multiply"></a>
Used for sucesive additions.

In [277]:
def multiply(constant, point = G):
    acu = add(point, point)
    for _ in range(constant-2):
        acu = add(acu, point)
    return acu

start = time()
result = multiply(40000)
print(f"Execution time: {(time() - start)*1000} ms")
print(f"Result: {{\n\tx: {result[0]},\n\ty: {result[1]}\n}}")

Execution time: 1976.719617843628 ms
Result: {
	x: 45144681238022915379838256640648974390095578890737782493692183842300544374804,
	y: 15493911184894630123643928078266102650003862558761004463646228136434723692345
}


In [278]:
def uber_multiply(constant, point = G):
    current = point
    binary = bin(constant)[3:]
    for i in binary:
        current = double(current)
        if i == '1':
            current = add(current, point)
    return current

start = time()
result = uber_multiply(40000)
print(f"Execution time: {(time() - start)*1000} ms")
print(f"Result: {{\n\tx: {result[0]},\n\ty: {result[1]}\n}}")

Execution time: 1.0192394256591797 ms
Result: {
	x: 45144681238022915379838256640648974390095578890737782493692183842300544374804,
	y: 15493911184894630123643928078266102650003862558761004463646228136434723692345
}


***
## ECDSA <a class="anchor" id="ecdsa"></a>

### Key Generation <a class="anchor" id="key-gen"></a>

The private key should be a random number less than orden 'n'.

In [279]:
d = 112757557418114203588093402336452206775565751179231977388358956335153294300646 # Pvivate key
Q = uber_multiply(d)

print("Public {}: {{\n\tx: {},\n\ty: {}\n}}".format("\U0001F511",Q[0], Q[1]))

Public 🔑: {
	x: 33886286099813419182054595252042348742146950914608322024530631065951421850289,
	y: 9529752953487881233694078263953407116222499632359298014255097182349749987176
}


### Sign<a class="anchor" id="sign"></a>

What we need:
1. Salt: $k$
2. Message Hash: $z$
3. Private Key: $d$

In [280]:
k = 12345
print(f"Salt: {k}")

message = "ECDSA is the most fun I have ever experienced"
print(f"Message: {message}")

z = sha256(message.encode('ascii')).digest()
z = int.from_bytes(z, 'big')
print(f"Hash digest in decimal: {z}")

print(f"Private key: {d}")

Salt: 12345
Message: ECDSA is the most fun I have ever experienced
Hash digest in decimal: 103318048148376957923607078689899464500752411597387986125144636642406244063093
Private key: 112757557418114203588093402336452206775565751179231977388358956335153294300646


A signature is compose by the conbination of $r$ and $s$:

$r = (G * k)mod(n)$</br>
We take the random number $k$ and multiply it by the generator point to get a random point $R$. We only actually use the x-coordinate of this point, and we call this lowercase $r$.

$s = k⁻¹ * (z + d * r)) mod(n)$</br>
This is a unique number created from a combination of $z$ and $d$, which is also bound to the random point using $r$.

In [281]:
def sign(private_k, m_hash, salt = None):

    if salt == None:
        salt = randbelow(n) # Random number within finite field order.

    r = uber_multiply(salt)[0] % n
   
    s = (inverse(salt, n) * (m_hash + private_k * r)) % n

    return (r, s)


signature = sign(d, z, k)

print(f"Signature: {{\n\tx: {signature[0]},\n\ty: {signature[1]}\n}}")

Signature: {
	x: 108607064596551879580190606910245687803607295064141551927605737287325610911759,
	y: 73791001770378044883749956175832052998232581925633570497458784569540878807131
}


### Bad signing practice<a class="anchor" id="bad-sign"></a>

In [288]:
prv_k = 1111222233334444555566667777888899990000
pub_k = uber_multiply(prv_k)

k = 800000
digest1 = int.from_bytes(sha256(b'Just a simple message.').digest(), 'big')
bad_sig1 = sign(prv_k, digest1, k)

digest2 = int.from_bytes(sha256(b'I have used the same k value.').digest(), 'big')
bad_sig2 = sign(prv_k, digest2, k)

k_recovery = ((digest1 - digest2) * inverse(bad_sig1[1] - bad_sig2[1], n)) % n

d_recovery = ((k_recovery * bad_sig1[1] - digest1) * inverse(bad_sig1[0], n)) % n

d_recovery

1111222233334444555566667777888899990000